In [ ]:
pip install -U lightautoml

In [ ]:
import os
import time
import re

#Necessary library
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,roc_auc_score
from sklearn.model_selection import train_test_split
import torch

#AUTOML
from lightautoml.automl.presets.tabular_presets import TabularAutoML,TabularUtilizedAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task

# Parameters and fix torch no of threads and numpy seeds

In [ ]:
N_THREADS = 4  #threads
RANDOM_STATE=256 # fixed random state
TIMEOUT = 8*3600 #Time

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Data Load

In [ ]:
%%time
train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
train.sample(3)

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")
sample=pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")
sample.sample(2)

# LightAutoML

# Create Metrics function

In [ ]:
%%time
def roc(ytrue,ypred,**kwargs):
    return roc_auc_score(ytrue,ypred,**kwargs)
task = Task('reg')

## features

In [ ]:
%%time

roles = {
    'target':'target',
    'drop':['id'],
}

# Train LightAutoML model on full data

In [ ]:
automl = TabularUtilizedAutoML(task = task,
                              timeout=TIMEOUT,
                              cpu_limit=N_THREADS,
                              reader_params = {'n_jobs':N_THREADS},
                              general_params = {'use_algos':[['linear_l2','lgb','lgb_tuned']]},
                              tuning_params = {'max_tuning_time':25*60},
                              lgb_params= {'default_params':{'num_trees':100000,'learning_rate':0.005,'early_stopping_rounds':1000},'freeze_defaults':True})

pred = automl.fit_predict(train,roles=roles)
print('pred:\n{}\nShape = {}'.format(pred[:10],pred.shape))


# **Predict test data**

In [ ]:
%%time
test_pred = automl.predict(test,n_jobs=N_THREADS)
print('prediction for test data:\n{}\nShape={}'.format(test_pred[:10],test_pred.shape))


In [ ]:
sample['target'] = test_pred.data[:, 0]
sample.to_csv('submission_lightAML.csv',index=False)

In [ ]:
sample.head()

## Reference:
1. [https://www.kaggle.com/alexryzhkov/lightautoml-tabulardata-love](http://) (**Full credit goes to this link**)
2. [https://github.com/sberbank-ai-lab/LightAutoML](http://)

## Thankyou Guys 🔥 🔥 🔥 